In [224]:
import json
import requests
from bs4 import BeautifulSoup as bs


url_data_path = '/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v2.json'

with open(url_data_path, 'r', encoding='utf-8') as f:
    urls = json.load(f)

In [210]:
API_BASE = 'https://s.search.naver.com/n/csearch/content/api/peopleworklistserviceapi.nhn?_callback=window.__jindo2_callback._page_0&pkid={pkid}&q=%EC%9D%B8%EB%AC%BC%EC%9E%91%ED%92%88%ED%99%9C%EB%8F%99&q_f1={q_f1}&display={display}&start={start}&CntPerPage=5'
#drama_thumbs = {}
#drama_contendsid = []
#drama_data = []
remove_keys = []

def get_soup(url):
    html = requests.get(url)
    assert html.status_code == 200, 'Response needs to be 200, but {}'.format(html)
    soup = bs(html.content, 'html.parser')
    
    return soup

def get_profile_url(soup):
    try:
        img = soup.find('img',{'class':'thmb_img'}).attrs['src']
    except:
        return None
    profile_url = img
    return profile_url

def update_thumb_url(nhn_json):
    drama_thumb = nhn_json['onair_image']
    drama_name = nhn_json['onair_keyword']
    drama_thumbs[drama_name] = drama_thumb


def get_params(soup):
    DISPLAY = 5
    START = 0
    PKID = 76
    
    params_list = []
    
    text = soup.find('div',{'class':'workact_dsc'}).text
    text = text.split('setOnairInfo(contentsid)')[1].split('oPagingListUi76')[0]
    text = text.split('if (')
    
    for t in text[1:]:
        params = {}
        contentsid = t.split(' ==')[0]
        
        q_f1 = contentsid.strip('\'')
        
        params['q_f1'] = q_f1
        params['display'] = DISPLAY
        params['start'] = START
        params['pkid'] = PKID
        
        if q_f1 not in drama_contendsid:
            params_list.append(params)
            drama_contendsid.append(q_f1)
    
    return params_list

def set_nhn(params):
    
    nhn_url = API_BASE.format(q_f1=params['q_f1'],
                    display=params['display'],
                    start=params['start'],
                    pkid=params['pkid']
                   )
    return nhn_url

def get_name(soup):
    soup = soup.find('div',{'class':'profile_dsc'})
    name = soup.find('dt',{'class':'name'}).text
    name = name.split('(')[0].strip()
    print(name)
    return name

def nhn_to_json(nhn):
    text = get_soup(nhn).text
    text = text.strip('window.__jindo2_callback._page_0(').strip(');')
    text = json.loads(text)['list'][0]
    drama_data.append(text)
    
    return text

def change_name(name, key):
    if name == key:
        return None
    else:
        if name in urls.keys():
            url_1 = urls[key]['actor_url']
            url_2 = urls[name]['actor_url']
            if url_1 == url_2:
                return None
            name = name + '_1'
        urls[name] = urls[key]
        remove_key.append(key)
        
def save(google_drive=True, local_drive=True):
    
    A_file = drama_contendsid
    B_file = drama_data
    C_file = drama_thumbs
    
    files = [A_file, B_file, C_file]
    
    A_file_name = 'drama_contendsid'
    B_file_name = 'drama_data'
    C_file_name = 'drama_thumbs'
    
    file_names = [A_file_name, B_file_name, C_file_name]
    

    for file, file_name in zip(files, file_names):
        if google_drive:
            with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/{}.json'.format(file_name), 'w', encoding='utf-8') as f:
                json.dump(file, f, indent='\t', ensure_ascii=False)
        
        if local_drive:
            with open('/Users/daeyeop/Work/KBS Drama project/Data/{}.json'.format(file_name) , 'w', encoding='utf-8') as f:
                json.dump(file, f, indent='\t', ensure_ascii=False)
            
        
def data_factory(key):
    actor_url = urls[key]['actor_url']
    soup = get_soup(actor_url)
    profile_img = get_profile_url(soup)
    urls[key]['profile_img'] = profile_img
    name = get_name(soup)
    change_name(name, key)
    params_list = get_params(soup)
    for params in params_list:
        nhn_url = set_nhn(params)
        nhn_json = nhn_to_json(nhn_url)
        update_thumb_url(nhn_json)
        save()
    
    return None

In [214]:
#keys = (key for key in urls.keys())
drama_thumbs, drama_data, drama_contendsid

({'루갈': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/29/77/40/57_10297740_poster_image_1583135689923.jpg',
  '싸이코패스다이어리': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/03/36/18/57_10033618_poster_image_1571722800543.jpg',
  '시크릿부티크': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/66/22/85/57_9662285_poster_image_1567403987204.jpg',
  '더뱅커': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/91/22/76/57_8912276_poster_image_1553478635132.jpg',
  '용왕님보우하사': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/74/45/03/57_8744503_poster_image_1546566251340.jpg',
  '무법변호사': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/25/02/72/57_7250272_poster_image_1525148318631.jpg',
  '돈꽃': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/22/36/28/57_6223628_poster_image_1510026530571.jpg',
  '내딸의남자들2아빠가보고있다': 'https://ssl.pstatic.net/sstatic/keypage/image/dss/57/36/86/67/57_6368667_poster_image_1508234015976.jpg',
  '드라마스페셜정마담의마지막일주일': 'https://ssl.pstatic.net

In [215]:
failed = []
for key in keys:
    try:
        data_factory(key)
    except:
        failed.append(key)
        print('Failed : ',key)

In [218]:
url_data_path = '/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v3.json'

with open(url_data_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent='\t', ensure_ascii=False)
    
google = '/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/drama_actors_v3.json'

with open(google, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent='\t', ensure_ascii=False)

In [217]:
key

'탤런트김수현'

In [222]:
len(urls.keys())

1934

In [232]:
url_data_path = '/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v3.json'
with open(url_data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [233]:
data.keys()

dict_keys(['박정학', '오민석', '정원중', '조이현', '채송아', '김지인', '박시후', '이응경', '유승목', '양해림', '한지완', '전혜원', '홍순창', '이선호', '김준원', '황영희', '김건우', '이고은', '최우혁', '안우연', '허성태', '옥예린', '이승우', '표예진', '김영철', '안정훈', '윤상화', '오혜원', '양형욱', '공형진', '진경', '송영규', '이무생', '배현성', '박솔미', '박상우', '정상훈', '김선경', '강지후', '윤은혜', '황정음', '김혜옥', '송재희', '안재현', '조한선', '정택현', '이용주', '배우희', '김영재', '지성', '천정명', '인교진', '이용녀', '이시아', '한수현', '백주희', '조경숙', '장석현', '서은아', '윤태영', '김지연', '최예빈', '정다은', '한가인', '박혜수', '안지훈', '박용우', '박서연', '윤송아', '심이영', '문현정', '전혜진', '배다빈', '안은진', '심민', '박은빈', '전익령', '박수빈', '이이경', '박종혁', '박형준', '조현식', '박민하', '김혜리', '김민경', '김지현', '최민', '최정윤', '박해준', '이주현', '최승경', '이용준', '제시', '김정영', '권율', '김성수', '김태우', '안소미', '이준영', '이정헌', '손현주', '권해효', '나혜진', '도지원', '주아성', '정하담', '이덕희', '변정수', '윤종훈', '류혜린', '김강우', '장도윤', '최찬호', '이태선', '김우석', '오종민', '최종환', '김수인', '정동환', '최다인', '이하늬', '박현숙', '정지소', '최성준', '최수종', '이현우', '이경진', '최윤라', '박성일', '최원영', '강지섭', '김환희', '이은재', '태진아', '최명길', '강상원', '박규영', '이찬형', '정수교', '이태연', '김호영', '임한빈', '

In [256]:
url_data_path = '/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v3.json'
with open(url_data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

url = 'https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query=탤런트김수현&os=99989&ie=utf8&key=PeopleService'

def info(url, key):
    soup = get_soup(url)
    soup = soup.find('dl',{'class':'dsc'})
    A = soup.find_all('dt')
    B = soup.find_all('dd')
    infomaion = {}
    for a,b in zip(A, B):
        infomaion[a.text] = b.text
    data[key]['info'] = infomaion

In [257]:
for key in data.keys():
    url = data[key]['actor_url']
    info(url, key)

In [255]:
data['박보검']

{'drama': {'청춘기록': {'role': '주연',
   'desc': '모델에서 배우로 전업 중. 현실적이고 실용적이다. 따뜻하면서도 선을 그어야 할 땐 확실하게 긋는다. 머리 좋고 공감 능력이 뛰어나다. 좋고 싫은 게 확실하지만 하고 싶은 일을 위해 싫은 것도 하는 유연성을 지녔다. 어릴 때부터 형과 비교 당하면서 설움도 많이 당했다.'},
  '드라마남자친구': {'role': '주연',
   'desc': '주민등록초본에 주소지 달랑 하나. 홍제동 집에서 태어나, 자랐고 여전히 살고 있다. 스물아홉 해 동안. 진혁의 아버지는 이 동네에서 진혁의 나이만큼 오랫동안 과일 가게를 하고 있다. 부모님을 닮아 착하고 이웃에 관심이 많은 진혁은 시장과 동네 어른들의 귀염둥이다. 다만, 공부는 좀 하던데 왜 졸업한 지 1년이 지나도록 취준생인지 모르겠다.'},
  '구르미그린달빛': {'role': '주연',
   'desc': '총명하고 아름다운 왕세자! 쇠락해가는 조선의 마지막 희망!......였었더랬다. 분명. 몇 년 전까지는. 어려서부터 학문을 즐거이 여겼고, 호기심이 많아 위험한 일에 몸을 쓰는 것을 두려워하지 않으며, 어린 나인의 볼멘소리까지 경청할 정도의 겸손함을 지닌 준비된 왕이나 다름없었는데, 그가 변했다. 언제부턴가.'},
  '응답하라1988': {'role': '주연',
   'desc': '천재 바둑 기사. 골목 금은방 ‘봉황당’집 외동아들. 바둑계의 돌부처로 불리는, 대한민국 국보급 바둑 기사다. 11살에 프로에 입단, 13살에 세계 최연소 타이틀 획득 이후, 88년 현재까지 바둑 랭킹 1위, 상금 1위의 자리를 지키며, 최고의 바둑 천재로 불린다. 불과 15세의 나이에 스승을 꺾는 파란을 일으키며 바둑계의 신흥 강자로 떠올랐다.'},
  '너를기억해': {'role': '조연',
   'desc': '20대 변호사. 힘 있고 돈 많은 자들을 위한 비윤리적 변호도 많이 하지만, 가끔씩 힘없는 범죄자나 재소자들을 위한 변호인 상담도 

In [ ]:
url_data_path = '/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v3.json'

with open(url_data_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent='\t', ensure_ascii=False)
    
google = '/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/drama_actors_v3.json'

with open(google, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent='\t', ensure_ascii=False)